# обучение

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/YOLO_model/DATASET.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Датасет распакован в: {extract_path}")
print(f"Содержимое папки: {os.listdir(extract_path)}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics

In [ ]:
# Пути к данным
DATASET_PATH = extract_path  # Основная папка датасета
IMAGES_PATH = os.path.join(DATASET_PATH, 'images')  # Папка с изображениями
LABELS_PATH = os.path.join(DATASET_PATH, 'labels')  # Папка с разметкой
CLASSES_FILE = os.path.join(DATASET_PATH, 'classes.txt')  # Файл с классами

подготовка датасета

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import yaml


def create_train_val_split():
    train_path = os.path.join('/content/dataset_splitted', 'train')
    val_path = os.path.join('/content/dataset_splitted', 'val')

    os.makedirs(os.path.join(train_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_path, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    all_images = [f for f in os.listdir(IMAGES_PATH) if f.endswith(('.jpg', '.jpeg', '.png'))]

    train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)

    # Копируем изображения и разметку в соответствующие папки
    for img in train_images:
        shutil.copy(os.path.join(IMAGES_PATH, img), os.path.join(train_path, 'images', img))

        label_file = os.path.splitext(img)[0] + '.txt'
        if os.path.exists(os.path.join(LABELS_PATH, label_file)):
            shutil.copy(os.path.join(LABELS_PATH, label_file), os.path.join(train_path, 'labels', label_file))

    for img in val_images:
        shutil.copy(os.path.join(IMAGES_PATH, img), os.path.join(val_path, 'images', img))

        label_file = os.path.splitext(img)[0] + '.txt'
        if os.path.exists(os.path.join(LABELS_PATH, label_file)):
            shutil.copy(os.path.join(LABELS_PATH, label_file), os.path.join(val_path, 'labels', label_file))

    return train_path, val_path


def create_data_yaml(train_path, val_path):
    with open(CLASSES_FILE, 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    data = {
        'train': os.path.join(train_path, 'images'),
        'val': os.path.join(val_path, 'images'),
        'nc': len(classes),
        'names': classes
    }

    yaml_path = os.path.join('/content/dataset_splitted', 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    return yaml_path


train_path, val_path = create_train_val_split()
print("Created train/val split")

yaml_path = create_data_yaml(train_path, val_path)
print(f"Created data.yaml at: {yaml_path}")

**Обучение модели**

In [ ]:
from ultralytics import YOLO
import torch
import os

# Проверка доступности GPU
print('CUDA available ->', torch.cuda.is_available())

# Основной процесс обучения
def main():

    # Загружаем модель
    model = YOLO('yolo11x.pt')

    # Параметры обучения
    results = model.train(
    data='/content/dataset_splitted/data.yaml',
    epochs=200,
    batch=8,           # Уменьшенный batch для T4
    imgsz=640,         # Можно снизить до 512 если не хватает памяти
    device='cuda',     # Используем GPU
    workers=2,         # 2 потока для загрузки данных
    lr0=0.01,          # Стандартная скорость обучения
    patience=30,       # Ранняя остановка при отсутствии улучшений
    augment=True,      # Включить аугментации
    hsv_h=0.015,       # Аугментация: оттенок
    fliplr=0.5,        # Аугментация: горизонтальное отражение
    dropout=0.1,       # Регуляризация для маленького датасета
    name='yolo_pleaseeeeeeeLAST'
    )

    # Сохраняем модель
    model.save('custom_yolo_model.pt')
    print("Training complete! Model saved as 'custom_yolo_model.pt'")

    # Валидация модели
    metrics = model.val()
    print(f"mAP50-95: {metrics.box.map} | mAP50: {metrics.box.map50}")

if __name__ == '__main__':
    main()

Тут просто скачивание архива со всеми результатми модели (путь указывать надо)

In [ ]:
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

# Путь к папке, которую нужно заархивировать
source_folder = '/content/runs'
# Путь для сохранения ZIP-архива
output_zip = '/content/yolo_lst.zip'

zip_folder(source_folder, output_zip)
print(f'Архив создан: {output_zip}')
files.download('/content/yolo_lst.zip')

In [ ]:
VAL_IMAGES_PATH = '/content/dataset_splitted/val/images'
MODEL_PATH = '/content/drive/MyDrive/YOLO_model/Almost there/custom_yolo_model.pt'  # Путь к лучшим весам

Тут можно прогнать через модель несколько изображений и вывести разметку

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np


# Загрузка модели
model = YOLO(MODEL_PATH)
print(f"Model loaded from {MODEL_PATH}")

# Получаем список изображений
val_images = [f for f in os.listdir(VAL_IMAGES_PATH)
             if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(val_images)
selected_images = val_images[:10]

# Настройка отображения
plt.figure(figsize=(20, 25))
for i, img_name in enumerate(selected_images):
    img_path = os.path.join(VAL_IMAGES_PATH, img_name)

    # Делаем предсказание
    results = model.predict(img_path, conf=0.5)
    result_img = results[0].plot(line_width=1, font_size=0.3)

    # Конвертируем BGR (OpenCV) в RGB (Matplotlib)
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)

    # Отображаем
    plt.subplot(5, 2, i+1)
    plt.imshow(result_img)
    plt.title(f"{img_name}", fontsize=1)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Вывод информации о классах
print("\nСоответствие ID классов и их имен:")
for class_id, class_name in model.names.items():
    print(f"{class_id}: {class_name}")